# Local analysis, screening Ecoinvent market consumption mixes: grouping by markets

In [2]:
import brightway2 as bw
import bw2calc as bc
import bw2data as bd
import numpy as np
import pandas as pd
import presamples as ps
import scipy
import scipy.stats as stats
from code.functions import make_dirichlet_distribution, get_elec_input, create_presamples, query_for_activities
ps.__version__

(0, 2, 6)

In [ ]:
from presamples import array

def custom_sample(self, index):
    """Draw a new sample from the pre-sampled arrays sequentially not randomly. The original code draws samples randomly. """
    index_ordered = self.count % self.data[0].shape[1]
    result = np.hstack([arr[:, index_ordered] for arr in self.data])
    self.count += 1   
    return result


array.RegularPresamplesArrays.sample = custom_sample

In [ ]:
# Generate objects for analysis

bw.projects.set_current("UK-wood-clca")

cutoff391 = bw.Database('cutoff-3.9.1')

fus = pd.read_excel(r"run.xlsx", sheet_name='to_run_2')
numbers = fus['indices'].tolist()
output_dir  = fus['names'].tolist()
amount  = fus['amount'].tolist()
reference_product  = fus['reference_product'].tolist()
tier = fus['tier'].tolist()


fu = [cutoff391.get({num}) for num in numbers]
methods = [('ReCiPe 2016 v1.03, midpoint (H)', 'climate change', 'global warming potential (GWP1000)'), ]

In [ ]:
Global_cat= pd.read_csv("results/Global__market_test_category_scores_60_products.csv")
reverse_mapping = {v: k for k, v in category_mapping.items()}

Global_cat['Category Number'] = Global_cat['names'].map(reverse_mapping)

fus = pd.read_excel(r"run.xlsx", sheet_name='to_run_2')
#numbers = pd.read_excel(r"presamples/to_run.xlsx", sheet_name='indices')
numbers = fus['indices'].tolist()
output_dir  = fus['names'].tolist()
amount  = fus['amount'].tolist()
amount  = fus['amount'].tolist()
reference_product  = fus['reference_product'].tolist()
tier = fus['tier'].tolist()

fu = [cutoff391.get({num}) for num in numbers]

In [ ]:
from code/morris_local_market_analysis import perform_local_morris_analysis

In [ ]:
%%time
Local_scores = []
for i in range(len(output_dir)):
    if len(Global_cat[Global_cat['fu'] == output_dir[i]]['names'].values) > 0: 
        sa_local = perform_local_morris_analysis(fu[i], 
                                                 1,
                                    output_dir[i], 
                                    output_dir[i],
                                     methods, 
                                     cutoff391, 
                                     trajectories = 10, 
                                     num_levels = 4,
                                    category_names=Global_cat[Global_cat['fu'] == output_dir[i]]['names'].values,
                                    category_numbers=Global_cat[Global_cat['fu'] == output_dir[i]]['Category Number'].values,
                                     market_dir= output_dir[i])
        sa_local['fu'] = output_dir[i]
        Local_scores.append(sa_local)
Local_scores = pd.concat(Local_scores, ignore_index=True)
Local_scores.to_csv(f"results/Local_scores_by_market_10.csv", index=False)